# Data driven approach


## monetray model +  yield curve + vix

multistep: multistep forecast  
q: quaterly

### dependent variable

- "DEXJPUS","fred"

### explanatory variables

- "MANMM101USM189S","fred"
- "MANMM101JPM189S","fred"
- "GDP","fred"
- "JPNNGDP","fred"
- "MKTGDPJPA646NWDB","fred"
- "USD12MD156N","fred"
- "JPY12MD156N","fred"
- "PPIACO","fred"
- "PITGCG01JPM661N","fred"
- "VXOCLS","fred"
- "TEDRATE","fred"

# multistep forecast
multistep: multistep forecast  
q: quartly
## mmyyrr ii sticky monetary approach +  vix

In [1]:
%matplotlib inline
import datetime
import pandas_datareader.data as web
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import VECM
start="1986/1/1";end='2019/12/31'
start1="1985/1/1";end1='2019/12/31'
from dateutil.relativedelta import relativedelta
from scipy.stats import norm
from scipy.stats import t
def tcdf(x,n): #t-distribution
    if x>=0:
        y=(1-t.cdf(x,n))
    else:
        y=t.cdf(x,n)
    return y
def ncdf(x): #t-distribution
    if x>=0:
        y=(1-norm.cdf(x))
    else:
        y=norm.cdf(x)
    return y

In [2]:
e =web.DataReader("DEXJPUS","fred",start,end)
m1 =web.DataReader("MANMM101USM189S","fred",start,end)/1000000000#w
m2 =web.DataReader("MANMM101JPM189S","fred",start,end)/1000000000#w
y1 =web.DataReader("GDP","fred",start,end)# norminal q
y2 =web.DataReader("JPNNGDP","fred",start,end)#nominal q
y3=web.DataReader("MKTGDPJPA646NWDB","fred",start,end)#q
r1 =web.DataReader("USD12MD156N","fred",start,end)#q
r2 =web.DataReader("JPY12MD156N","fred",start,end)#q
p3 =web.DataReader("PPIACO","fred",start1,end)#m
p4 =web.DataReader("PITGCG01JPM661N","fred",start1,end)#m
v =web.DataReader("VXOCLS","fred",start,end)#m
t =web.DataReader("TEDRATE","fred",start,end)#m
#y1 =web.DataReader("GDPC1","fred",start,end)# real q
#y2 =web.DataReader("JPNRGDPEXP","fred",start,end)# real

In [3]:
FRQ=['QS-JAN']
interval=3
for i in range(len(FRQ)):
    ee=e.copy().resample(FRQ[i]).last()
    ee.columns=['x']
    mm1=m1.copy().resample(FRQ[i]).last()
    mm1.columns=['x']
    mm2=m2.copy().resample(FRQ[i]).last()
    mm2.columns=['x']
    yy1=y1.copy().resample(FRQ[i]).last()
    yy1.columns=['x']
    yy2=y2.copy().resample(FRQ[i]).last()
    yy2.columns=['x']
    rr1=r1.copy().resample(FRQ[i]).last()
    rr1.columns=['x']
    rr2=r2.copy().resample(FRQ[i]).last()
    rr2.columns=['x']
    pp1=(p3.copy().pct_change().rolling(12).mean().dropna()).resample(FRQ[i]).last()
    pp1.columns=['x']
    pp2=(p4.copy().pct_change().rolling(12).mean().dropna()).resample(FRQ[i]).last()
    pp2.columns=['x']
    vv=v.copy().resample(FRQ[i]).last()
    vv.columns=['x']
    tt=t.copy().resample(FRQ[i]).last()
    tt.columns=['x']

    tsd=pd.concat([ee,mm1,mm2,yy1,yy2,rr1,rr2,pp1,pp2,vv,tt],axis=1)
    tsd.columns=['ee','m1','m2','y1','y2','r1','r2','p1','p2','vix','ted']
    #tsd=tsd.fillna(0)
    idx=e.index[-1]-relativedelta(months=2)
    tsd=tsd.copy().loc[:idx]
    print(i,FRQ[i],tsd.tail(5))
    if i==0:
        tsd1=tsd
    elif i==1:
        tsd2=tsd
    else:
        tsd3=tsd

0 QS-JAN                 ee      m1        m2         y1        y2       r1       r2  \
DATE                                                                          
2018-10-01  109.70  3746.5  772898.2  20897.804  546737.7  3.00544  0.10633   
2019-01-01  110.68  3730.0  783525.1  21098.827  552687.8  2.71063  0.09250   
2019-04-01  107.84  3832.9  793923.2  21340.267  555954.0  2.17813  0.04567   
2019-07-01  108.11  3903.3  804952.1  21542.540  558237.1  2.03213  0.02550   
2019-10-01  108.67  3978.4  817294.3  21729.124  549920.9  1.99625  0.10483   

                  p1        p2    vix   ted  
DATE                                         
2018-10-01  0.001996 -0.001016  28.18  0.41  
2019-01-01  0.000651 -0.000329  14.02  0.25  
2019-04-01 -0.001588 -0.001102  16.03  0.24  
2019-07-01 -0.002134 -0.001708  17.10  0.25  
2019-10-01 -0.000736 -0.000939  13.31  0.37  


In [4]:
y4=y3.resample('QS').asfreq()
y4=y4.interpolate()/1000000000
y5=pd.concat([tsd1.ee,y4,y2],axis=1)
y5.columns=['e','y4','y2']
y5.iloc[:,1]=y5.y4*y5.e
y6=(y5.y4.rolling(window=8).mean()).shift(-6)
y6=y6.bfill()
l=len(y5.loc['1994':])
y6.iloc[l:]=y5.iloc[l:,2].values
tsd1=pd.concat([ee,mm1,mm2,yy1,y6,rr1,rr2,pp1,pp2,vv,tt],axis=1)
tsd1.columns=['ee','m1','m2','y1','y2','r1','r2','p1','p2','vix','ted']
tsd=tsd1

In [5]:
tsd.head(),tsd.tail()

(                ee     m1             m2        y1             y2      r1  \
 DATE                                                                        
 1986-01-01  177.60  633.5  105560.846100  4507.894  374305.440921  7.4375   
 1986-04-01  163.80  660.6  108346.389192  4545.340  374305.440921  6.9375   
 1986-07-01  154.35  687.4  110798.316365  4607.669  375802.159656  6.3125   
 1986-10-01  158.30  724.7  113932.083556  4657.627  382112.674481  6.3125   
 1987-01-01  145.68  733.8  115856.992469  4722.156  388895.543982  6.8125   
 
                 r2        p1        p2    vix   ted  
 DATE                                                 
 1986-01-01  5.1875 -0.002270 -0.001197  23.33  1.10  
 1986-04-01  4.7344 -0.002763 -0.001403  18.31  0.92  
 1986-07-01  4.9063 -0.002208 -0.002150  24.76  0.92  
 1986-10-01  4.4375 -0.003176 -0.002158  18.71  0.77  
 1987-01-01  4.0625  0.000753 -0.002103  23.03  1.08  ,
                 ee      m1        m2         y1        y2       r

In [8]:
def backtest_ols_dp(tsd,tw,pt):#tw time window pt:predicting time
    pred=[]
    real0=[]
    real9=[]
    tsd0=np.log(tsd.iloc[:,:6]).copy()
    for i in range(tw+1,len(tsd)+1-1):
        yy=tsd0.iloc[i-tw:i:1,0]
        y0=tsd0.iloc[i-tw-1:i:1,0].diff().dropna()
        x0=(tsd0.iloc[i-tw-1:i:1,1]-tsd0.iloc[i-tw-1:i:1,2]).diff().dropna()
        x00=(tsd0.iloc[i-tw-1:i:1,3]-tsd0.iloc[i-tw-1:i:1,4]).diff().dropna()
        x000=((1+tsd.iloc[i-tw-1:i:1,6]/100)/(1+tsd.iloc[i-tw-1:i:1,5]/100)).diff().dropna()
        x0000=((1+tsd.iloc[i-tw-1:i:1,8])/(1+tsd.iloc[i-tw-1:i:1,7])).diff().dropna()
        xx=tsd.iloc[i-tw-1:i:1,9].diff().dropna()
        xxx=tsd.iloc[i-tw-1:i:1,10].diff().dropna()
        x0=pd.concat([x0,x00,x000,x0000,xx,xxx],axis=1)
        x0=sm.add_constant(x0.values)
        res= sm.OLS(y0, x0).fit()
        real0.append(np.exp(tsd0.iloc[i-1:i,0][-1]))
        if i<len(tsd)+1-pt:
            x9=(tsd0.iloc[i-1:i+pt:1,1]-tsd0.iloc[i-1:i+pt:1,2]).diff().dropna()
            x99=(tsd0.iloc[i-1:i+pt:1,3]-tsd0.iloc[i-1:i+pt:1,4]).diff().dropna()
            x999=((1+tsd.iloc[i-1:i+pt:1,6]/100)/(1+tsd.iloc[i-1:i+pt:1,5]/100)).diff().dropna()
            x9999=((1+tsd.iloc[i-1:i+pt:1,8])/(1+tsd.iloc[i-1:i+pt:1,7])).diff().dropna()
            xx9=(tsd.iloc[i-1:i+pt:1,9]).diff().dropna()
            xxx9=(tsd.iloc[i-1:i+pt:1,10]).diff().dropna()
            x9=pd.concat([x9,x99,x999,x9999,xx9,xxx9],axis=1)
            #print(x9)
            if pt==1:
                x9=np.insert(x9.values,0,1)
            else:
                x9=sm.add_constant(x9.values)
            pred0=res.predict(x9)[-1:]
            pred.append(np.exp(pred0+yy.iloc[-1]))
            real9.append(np.exp(tsd0.iloc[i+pt-1:i+pt:1,0].values))
    return real0,real9,pred

def performance_eval(real0,real9,pred):
    p0=pd.DataFrame(real0)
    p9=pd.DataFrame(real9)
    pp=pd.DataFrame(pred)
    p0_=p0[:len(p9)]
    pp_=pp[:len(p9)]
    wrs=p0_[(p9<p0_)&(pp_<p0_)].count()
    wrl=p0_[(p9>p0_)&(pp_>p0_)].count()
    wr=float((wrs+wrl)/p0_.count())
    #wrp=tcdf(float((wr-0.5)/np.sqrt(0.25/p0_.count())),p0_.count())[0]
    wrp=ncdf(float((wr-0.5)/np.sqrt(0.25/p0_.count())))
    d_x=(pp_-p9)**2
    d_rw=(p9-p0_)**2
    d=d_x-d_rw
    d_=float(d.mean())
    #dm=tcdf(float(d_/np.sqrt(((d-d_)**2).sum()/len(d)**2)),p0_.count())[0]
    dm=ncdf(float(d_/np.sqrt(((d-d_)**2).sum()/len(d)**2)))
    rmse=float(d_x.mean()/d_rw.mean())
    return rmse,dm,wr,wrp

def performance_report(real0,real9,pred):
    ab,dm,wr,wrp=performance_eval(real0,real9,pred)
    print('{0:20s}{1:5d} mse_r {2:7.4f} mse_p {3:7.2f} wr {4:7.4f} wrp {5:7.2f} len {6:4d}'
          .format(model,isp,ab,dm,wr,wrp,len(real9)))    

In [9]:
model='OLS'
oosp=1
pstart=datetime.date(1991,1,1)
pend=datetime.date(1994,12,31)        
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*10,oosp*50,3):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    10 mse_r  3.4818 mse_p    0.13 wr  0.7857 wrp    0.02 len   14
OLS                    13 mse_r  2.6424 mse_p    0.13 wr  0.5000 wrp    0.50 len   14
OLS                    16 mse_r  2.7142 mse_p    0.10 wr  0.5000 wrp    0.50 len   14
OLS                    19 mse_r  2.2734 mse_p    0.16 wr  0.6429 wrp    0.14 len   14


In [10]:
model='OLS'
oosp=1
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)        
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*10,oosp*50,3):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    10 mse_r  3.2580 mse_p    0.00 wr  0.5800 wrp    0.13 len   50
OLS                    13 mse_r  2.6436 mse_p    0.00 wr  0.4800 wrp    0.39 len   50
OLS                    16 mse_r  1.7815 mse_p    0.02 wr  0.5600 wrp    0.20 len   50
OLS                    19 mse_r  1.5358 mse_p    0.09 wr  0.5600 wrp    0.20 len   50
OLS                    22 mse_r  1.5632 mse_p    0.12 wr  0.5600 wrp    0.20 len   50
OLS                    25 mse_r  1.2303 mse_p    0.18 wr  0.5600 wrp    0.20 len   50
OLS                    28 mse_r  1.2017 mse_p    0.21 wr  0.6200 wrp    0.04 len   50
OLS                    31 mse_r  1.1109 mse_p    0.28 wr  0.6200 wrp    0.04 len   50
OLS                    34 mse_r  1.0734 mse_p    0.33 wr  0.6000 wrp    0.08 len   50
OLS                    37 mse_r  1.0832 mse_p    0.28 wr  0.5600 wrp    0.20 len   50


In [11]:
model='OLS'
oosp=1
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)        
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*10,oosp*50,3):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    10 mse_r  4.8269 mse_p    0.00 wr  0.4783 wrp    0.38 len   46
OLS                    13 mse_r  3.1806 mse_p    0.00 wr  0.6087 wrp    0.07 len   46
OLS                    16 mse_r  2.2604 mse_p    0.00 wr  0.5652 wrp    0.19 len   46
OLS                    19 mse_r  1.6192 mse_p    0.03 wr  0.5870 wrp    0.12 len   46
OLS                    22 mse_r  1.4074 mse_p    0.02 wr  0.5652 wrp    0.19 len   46
OLS                    25 mse_r  1.4042 mse_p    0.03 wr  0.5870 wrp    0.12 len   46
OLS                    28 mse_r  1.6157 mse_p    0.02 wr  0.5652 wrp    0.19 len   46
OLS                    31 mse_r  1.6510 mse_p    0.06 wr  0.6304 wrp    0.04 len   46
OLS                    34 mse_r  1.5211 mse_p    0.05 wr  0.5870 wrp    0.12 len   46
OLS                    37 mse_r  1.6630 mse_p    0.09 wr  0.5652 wrp    0.19 len   46
OLS                    40 mse_r  1.7016 mse_p    0.07 wr  0.5652 wrp    0.19 len   46
OLS                    43 mse_r  1.6063 mse_p    0.09 

In [12]:
model='OLS'
oosp=4
pstart=datetime.date(1991,1,1)
pend=datetime.date(1994,12,31)       
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*3,oosp*50,3):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    12 mse_r  1.0815 mse_p    0.39 wr  0.6364 wrp    0.18 len   11
OLS                    15 mse_r  1.0141 mse_p    0.47 wr  0.5455 wrp    0.38 len   11
OLS                    18 mse_r  0.9112 mse_p    0.36 wr  0.8182 wrp    0.02 len   11
OLS                    21 mse_r  0.7582 mse_p    0.06 wr  1.0000 wrp    0.00 len   11


In [13]:
model='OLS'
oosp=4
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)       
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*3,oosp*50,6):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    12 mse_r  2.6779 mse_p    0.05 wr  0.4043 wrp    0.09 len   47
OLS                    18 mse_r  1.0636 mse_p    0.31 wr  0.5319 wrp    0.33 len   47
OLS                    24 mse_r  1.0610 mse_p    0.34 wr  0.5532 wrp    0.23 len   47
OLS                    30 mse_r  1.0496 mse_p    0.33 wr  0.5106 wrp    0.44 len   47
OLS                    36 mse_r  0.8871 mse_p    0.12 wr  0.5957 wrp    0.09 len   47


In [14]:
model='OLS'
oosp=4
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)       
nn=len(tsd.loc[pstart:pend])
for isp in range(oosp*3,oosp*50,6):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    12 mse_r  2.2775 mse_p    0.00 wr  0.4651 wrp    0.32 len   43
OLS                    18 mse_r  1.1812 mse_p    0.06 wr  0.5116 wrp    0.44 len   43
OLS                    24 mse_r  1.0355 mse_p    0.37 wr  0.6047 wrp    0.08 len   43
OLS                    30 mse_r  1.0883 mse_p    0.21 wr  0.5581 wrp    0.22 len   43
OLS                    36 mse_r  1.1143 mse_p    0.11 wr  0.5116 wrp    0.44 len   43
OLS                    42 mse_r  1.0460 mse_p    0.26 wr  0.5349 wrp    0.32 len   43
OLS                    48 mse_r  1.0059 mse_p    0.46 wr  0.5349 wrp    0.32 len   43
OLS                    54 mse_r  0.9754 mse_p    0.35 wr  0.5581 wrp    0.22 len   43
OLS                    60 mse_r  0.9536 mse_p    0.24 wr  0.6047 wrp    0.08 len   43
OLS                    66 mse_r  0.8976 mse_p    0.07 wr  0.6279 wrp    0.05 len   43
OLS                    72 mse_r  0.8892 mse_p    0.05 wr  0.5349 wrp    0.32 len   43
OLS                    78 mse_r  0.9183 mse_p    0.12 

In [15]:
model='OLS'
oosp=20
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)   
nn=len(tsd.loc[pstart:pend])        
for isp in range(12,oosp*10,2):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    12 mse_r  2.3788 mse_p    0.00 wr  0.4839 wrp    0.43 len   31
OLS                    14 mse_r  1.7037 mse_p    0.00 wr  0.5484 wrp    0.30 len   31
OLS                    16 mse_r  1.4297 mse_p    0.04 wr  0.5806 wrp    0.18 len   31
OLS                    18 mse_r  1.2062 mse_p    0.14 wr  0.5806 wrp    0.18 len   31
OLS                    20 mse_r  1.3950 mse_p    0.04 wr  0.5484 wrp    0.30 len   31
OLS                    22 mse_r  1.3298 mse_p    0.06 wr  0.5806 wrp    0.18 len   31
OLS                    24 mse_r  1.3818 mse_p    0.05 wr  0.5161 wrp    0.43 len   31
OLS                    26 mse_r  1.2179 mse_p    0.15 wr  0.5484 wrp    0.30 len   31
OLS                    28 mse_r  0.9710 mse_p    0.42 wr  0.5806 wrp    0.18 len   31
OLS                    30 mse_r  0.8878 mse_p    0.20 wr  0.5806 wrp    0.18 len   31
OLS                    32 mse_r  0.8068 mse_p    0.07 wr  0.6452 wrp    0.05 len   31
OLS                    34 mse_r  0.8496 mse_p    0.12 

In [16]:
model='OLS'
oosp=20
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)   
nn=len(tsd.loc[pstart:pend])        
for isp in range(15,oosp*10,12):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_ols_dp(tsdd,isp,oosp)
        performance_report(real0,real9,pred)

OLS                    15 mse_r  0.9557 mse_p    0.18 wr  0.6296 wrp    0.09 len   27
OLS                    27 mse_r  0.9218 mse_p    0.00 wr  0.6667 wrp    0.04 len   27
OLS                    39 mse_r  0.9574 mse_p    0.03 wr  0.7037 wrp    0.02 len   27
OLS                    51 mse_r  0.9435 mse_p    0.03 wr  0.7037 wrp    0.02 len   27
OLS                    63 mse_r  0.9295 mse_p    0.01 wr  0.6667 wrp    0.04 len   27
OLS                    75 mse_r  0.9572 mse_p    0.04 wr  0.6296 wrp    0.09 len   27
OLS                    87 mse_r  0.9688 mse_p    0.07 wr  0.6296 wrp    0.09 len   27


In [17]:
def backtest_VECM(tsd,tw,pt,det):#tw time window pt:predicting time
    pred=[]
    real0=[]
    real9=[]
    tsd0=np.log(tsd.iloc[:,:6]).copy()    
    for i in range(tw,len(tsd)+1):
        y0=(tsd0.iloc[i-tw:i:1,0])
        x=tsd0.iloc[i-tw:i:1,1]-tsd0.iloc[i-tw:i:1,2]
        xx=tsd0.iloc[i-tw:i:1,3]-tsd0.iloc[i-tw:i:1,4]
        xxx=(1+tsd.iloc[i-tw:i:1,6]/100)/(1+tsd.iloc[i-tw:i:1,5]/100)
        xxxx=(1+tsd.iloc[i-tw:i:1,8])/(1+tsd.iloc[i-tw:i:1,7])
        x0=(tsd.iloc[i-tw:i:1,9])
        xx0=(tsd.iloc[i-tw:i:1,10])
        tsd00=pd.concat([y0,x,xx,xxx,x0,xx0],axis=1)
        model = VECM(tsd00,k_ar_diff=0,coint_rank=1,deterministic=det)
        results = model.fit() 
        real0.append(np.exp(y0.iloc[-1]))
        pp=np.exp(results.predict(pt)[-1][0])
        pred.append(pp)
        if i<len(tsd)+1-pt:
            real9.append(tsd.iloc[i+pt-1,0])
    return real0,real9,pred

In [18]:
model='VECM'
oosp=1
pstart=datetime.date(1991,1,1)
pend=datetime.date(1994,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*10,oosp*200,oosp*1):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'li')
            performance_report(real0,real9,pred)

VECM                   10 mse_r  2.0586 mse_p    0.05 wr  0.5333 wrp    0.40 len   15
VECM                   11 mse_r  1.1035 mse_p    0.35 wr  0.6000 wrp    0.22 len   15
VECM                   12 mse_r  0.8940 mse_p    0.30 wr  0.7333 wrp    0.04 len   15
VECM                   13 mse_r  1.7738 mse_p    0.09 wr  0.3333 wrp    0.10 len   15
VECM                   14 mse_r  1.2198 mse_p    0.20 wr  0.6000 wrp    0.22 len   15
VECM                   15 mse_r  1.1042 mse_p    0.32 wr  0.4000 wrp    0.22 len   15
VECM                   16 mse_r  0.9469 mse_p    0.34 wr  0.4000 wrp    0.22 len   15
VECM                   17 mse_r  1.1850 mse_p    0.06 wr  0.5333 wrp    0.40 len   15
VECM                   18 mse_r  1.1920 mse_p    0.19 wr  0.4000 wrp    0.22 len   15
VECM                   19 mse_r  0.9689 mse_p    0.45 wr  0.6000 wrp    0.22 len   15
VECM                   20 mse_r  0.8187 mse_p    0.25 wr  0.8000 wrp    0.01 len   15
VECM                   21 mse_r  1.2023 mse_p    0.23 

In [19]:
model='VECM'
oosp=1
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)   
nn=len(tsd.loc[pstart:pend])    
for isp in range(oosp*18,oosp*200,oosp*2):#10,pt):
    ps=pstart-relativedelta(months=isp*interval-1)
    tsdd=tsd.loc[ps:pend]
    if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
        real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
        performance_report(real0,real9,pred)

VECM                   18 mse_r  1.1142 mse_p    0.10 wr  0.4314 wrp    0.16 len   51
VECM                   20 mse_r  1.0597 mse_p    0.24 wr  0.5490 wrp    0.24 len   51
VECM                   22 mse_r  1.0709 mse_p    0.19 wr  0.6471 wrp    0.02 len   51
VECM                   24 mse_r  1.0741 mse_p    0.14 wr  0.6078 wrp    0.06 len   51
VECM                   26 mse_r  1.0712 mse_p    0.15 wr  0.5882 wrp    0.10 len   51
VECM                   28 mse_r  1.1619 mse_p    0.05 wr  0.4706 wrp    0.34 len   51
VECM                   30 mse_r  1.1864 mse_p    0.01 wr  0.3922 wrp    0.06 len   51
VECM                   32 mse_r  1.1049 mse_p    0.03 wr  0.4510 wrp    0.24 len   51
VECM                   34 mse_r  1.1104 mse_p    0.01 wr  0.3922 wrp    0.06 len   51
VECM                   36 mse_r  1.0238 mse_p    0.27 wr  0.4902 wrp    0.44 len   51


In [20]:
model='VECM'
oosp=1
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*12,oosp*200,oosp*12):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
            performance_report(real0,real9,pred)

VECM                   12 mse_r  1.7398 mse_p    0.04 wr  0.5106 wrp    0.44 len   47
VECM                   24 mse_r  1.2795 mse_p    0.07 wr  0.5319 wrp    0.33 len   47
VECM                   36 mse_r  1.0130 mse_p    0.40 wr  0.4681 wrp    0.33 len   47
VECM                   48 mse_r  1.0057 mse_p    0.41 wr  0.4255 wrp    0.15 len   47
VECM                   60 mse_r  1.0112 mse_p    0.28 wr  0.4894 wrp    0.44 len   47
VECM                   72 mse_r  0.9939 mse_p    0.28 wr  0.4894 wrp    0.44 len   47
VECM                   84 mse_r  1.0098 mse_p    0.09 wr  0.5319 wrp    0.33 len   47


In [21]:
model='VECM'
oosp=4
pstart=datetime.date(1991,1,1)
pend=datetime.date(1995,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*2,oosp*200,oosp*1):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
            performance_report(real0,real9,pred)

VECM                    8 mse_r 123.2858 mse_p    0.14 wr  0.4375 wrp    0.31 len   16
VECM                   12 mse_r  0.8673 mse_p    0.12 wr  0.7500 wrp    0.02 len   16
VECM                   16 mse_r  0.7332 mse_p    0.00 wr  0.8125 wrp    0.01 len   16
VECM                   20 mse_r  0.5496 mse_p    0.00 wr  0.8750 wrp    0.00 len   16


In [22]:
model='VECM'
oosp=4
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*2,oosp*200,oosp*1):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
            performance_report(real0,real9,pred)

VECM                    8 mse_r 13762546927.2223 mse_p    0.16 wr  0.4583 wrp    0.28 len   48
VECM                   12 mse_r  1.3550 mse_p    0.08 wr  0.4583 wrp    0.28 len   48
VECM                   16 mse_r  1.4517 mse_p    0.01 wr  0.5000 wrp    0.50 len   48
VECM                   20 mse_r  1.2649 mse_p    0.08 wr  0.5208 wrp    0.39 len   48
VECM                   24 mse_r  1.1767 mse_p    0.11 wr  0.5208 wrp    0.39 len   48
VECM                   28 mse_r  1.3850 mse_p    0.01 wr  0.3333 wrp    0.01 len   48
VECM                   32 mse_r  1.4171 mse_p    0.00 wr  0.3333 wrp    0.01 len   48
VECM                   36 mse_r  1.2507 mse_p    0.00 wr  0.3750 wrp    0.04 len   48


In [23]:
model='VECM'
oosp=4
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*6,oosp*200,oosp*2):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
            performance_report(real0,real9,pred)

VECM                   24 mse_r  1.4166 mse_p    0.01 wr  0.5455 wrp    0.27 len   44
VECM                   32 mse_r  1.0457 mse_p    0.27 wr  0.5000 wrp    0.50 len   44
VECM                   40 mse_r  1.0767 mse_p    0.17 wr  0.5455 wrp    0.27 len   44
VECM                   48 mse_r  0.9970 mse_p    0.47 wr  0.4545 wrp    0.27 len   44
VECM                   56 mse_r  1.0250 mse_p    0.19 wr  0.5909 wrp    0.11 len   44
VECM                   64 mse_r  1.0009 mse_p    0.48 wr  0.3409 wrp    0.02 len   44
VECM                   72 mse_r  0.9525 mse_p    0.01 wr  0.5455 wrp    0.27 len   44
VECM                   80 mse_r  0.9837 mse_p    0.20 wr  0.5227 wrp    0.38 len   44
VECM                   88 mse_r  1.0174 mse_p    0.08 wr  0.4318 wrp    0.18 len   44


In [24]:
model='VECM'
oosp=20
pstart=datetime.date(1995,1,1)
pend=datetime.date(2007,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*1,oosp*50,2):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
            performance_report(real0,real9,pred)

VECM                   20 mse_r  8.1979 mse_p    0.05 wr  0.5312 wrp    0.36 len   32
VECM                   22 mse_r  7.5010 mse_p    0.01 wr  0.5312 wrp    0.36 len   32
VECM                   24 mse_r  6.5061 mse_p    0.01 wr  0.5312 wrp    0.36 len   32
VECM                   26 mse_r  6.4570 mse_p    0.01 wr  0.4688 wrp    0.36 len   32
VECM                   28 mse_r  7.3818 mse_p    0.00 wr  0.3438 wrp    0.04 len   32
VECM                   30 mse_r  7.8243 mse_p    0.00 wr  0.2500 wrp    0.00 len   32
VECM                   32 mse_r  9.9880 mse_p    0.01 wr  0.3125 wrp    0.02 len   32
VECM                   34 mse_r  7.9480 mse_p    0.02 wr  0.1875 wrp    0.00 len   32
VECM                   36 mse_r  4.3335 mse_p    0.05 wr  0.3750 wrp    0.08 len   32


In [25]:
model='VECM'
oosp=20
pstart=datetime.date(2008,1,1)
pend=datetime.date(2020,12,31)   
nn=len(tsd.loc[pstart:pend])    
for f in range(len(FRQ)):
    for isp in range(oosp*2,oosp*60,6):#10,pt):
        ps=pstart-relativedelta(months=isp*interval-1)
        tsdd=tsd.loc[ps:pend]
        if (len(tsdd)-isp==nn) or (len(tsdd)-isp==nn-1):
            real0,real9,pred=backtest_VECM(tsdd,isp,oosp,'ci')
            performance_report(real0,real9,pred)

VECM                   40 mse_r  1.2247 mse_p    0.00 wr  0.2857 wrp    0.01 len   28
VECM                   46 mse_r  1.1232 mse_p    0.00 wr  0.3571 wrp    0.07 len   28
VECM                   52 mse_r  1.0974 mse_p    0.03 wr  0.6429 wrp    0.07 len   28
VECM                   58 mse_r  1.0625 mse_p    0.07 wr  0.5714 wrp    0.22 len   28
VECM                   64 mse_r  0.9740 mse_p    0.15 wr  0.6071 wrp    0.13 len   28
VECM                   70 mse_r  0.9539 mse_p    0.00 wr  0.8214 wrp    0.00 len   28
VECM                   76 mse_r  0.9881 mse_p    0.23 wr  0.7143 wrp    0.01 len   28
VECM                   82 mse_r  1.0190 mse_p    0.15 wr  0.5714 wrp    0.22 len   28
VECM                   88 mse_r  0.9897 mse_p    0.24 wr  0.6786 wrp    0.03 len   28
